In [1]:
import os
os.chdir("../..")

In [2]:
# Use transformer previous version
import torch
import json
from tqdm import tqdm
from pathlib import Path
from transformers import RobertaForSequenceClassification, RobertaTokenizer

state_path = Path("data", "checkpoint", "solaiman-detector-base.pt")
model = RobertaForSequenceClassification.from_pretrained("roberta-base")
tokenizer = RobertaTokenizer.from_pretrained("roberta-base", truncation=True, max_length=510)

model.load_state_dict(torch.load(state_path)["model_state_dict"])
model.to("cuda")


RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): L

In [3]:
def load_line(content):
    entry = json.loads(content)
    return {"text": entry["text"], "source": entry["extra"]["source"]}


def evaluate_entry(entry, model):
    text, src = entry["text"], entry["source"]

    # encode
    tokens = tokenizer.encode(text, max_length=512)
    tokens = tokens[:tokenizer.max_len - 2]
    tokens = torch.tensor([tokenizer.bos_token_id] + tokens + [tokenizer.eos_token_id]).unsqueeze(0)
    mask = torch.ones_like(tokens)

    # forward propagation
    with torch.no_grad():
        logits = model(tokens.to("cuda"), attention_mask=mask.to("cuda"))[0]
        probs = logits.softmax(dim=-1)

    # update statistics
    generated, human = probs.detach().cpu().flatten().numpy().tolist()

    return {"pred": [generated, human], "source": src}


In [4]:
def evaluate_files(from_files, to_files, model):
    for from_file, to_file in zip(from_files, to_files):
        with open(Path("data", "split", from_file), "r") as f:
            content = f.read().strip().split("\n")

        with open(Path("data", "baselines", "solaiman", to_file), "w") as f:
            for line in tqdm(content):
                entry = load_line(line)
                try:
                    result = evaluate_entry(entry, model)
                    f.write(json.dumps(result) + "\n")
                except:
                    print("Failed to process", entry)
                    pass


In [5]:
evaluate_files(
    [
        # "open-web-text/test.jsonl", "open-gpt-text/test.jsonl", 
        "open-palm-text/test.jsonl", "open-llama-text/test.jsonl", "gpt2-output/test.jsonl"
    ],
    [
        # "solaiman-openweb.jsonl", "solaiman-opengpt.jsonl",
        "solaiman-openpalm.jsonl", "solaiman-openllama.jsonl", "solaiman-gpt2.jsonl"
    ],
    model
)

 35%|███▍      | 2583/7400 [00:51<01:20, 59.81it/s]

Failed to process {'text': '', 'source': 'palm'}


 93%|█████████▎| 6877/7400 [02:12<00:08, 60.75it/s]

Failed to process {'text': '', 'source': 'palm'}


100%|██████████| 7385/7385 [03:03<00:00, 40.33it/s]
